# Heart Disease Risk Prediction: Logistic Regression Homework

**Student Name:** [Tu Nombre]  
**Date:** January 28, 2026

## Introduction

Heart disease is the world's leading cause of death, claiming approximately 18 million lives each year, as reported by the World Health Organization. Predictive models like logistic regression can enable early identification of at-risk patients by analyzing clinical features such as age, cholesterol, and blood pressure.

This notebook implements **logistic regression** on the **Heart Disease Dataset** from Kaggle—a real-world UCI repository collection of 303 patient records with 14 features and a binary target (1 for disease presence, 0 for absence).

### Dataset Source
- **Kaggle:** https://www.kaggle.com/datasets/neurocipher/heartdisease
- **Description:** 303 patient records with 14 clinical features
- **Target:** Heart Disease (Presence/Absence)

---

# Step 1: Load and Prepare the Dataset

In this section, we will:
1. Load the Heart Disease dataset from CSV
2. Perform Exploratory Data Analysis (EDA)
3. Handle missing values and outliers
4. Binarize the target variable
5. Select relevant features
6. Split data into train/test sets (70/30, stratified)
7. Normalize numerical features

---

## 1.1 Import Required Libraries

In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 1.2 Load the Dataset

In [ ]:
# Load the Heart Disease dataset
df = pd.read_csv('Heart_Disease_Prediction.csv')

# Display basic information
print("Dataset loaded successfully!")
print(f"\nDataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print("\n" + "="*60)
print("First 5 rows:")
print("="*60)
df.head()

In [ ]:
# Display all column names
print("Column Names:")
print("="*60)
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

## 1.3 Initial Data Exploration

Let's examine the dataset structure, data types, and summary statistics.

In [ ]:
# Display dataset info
print("Dataset Information:")
print("="*60)
df.info()
print("\n" + "="*60)
print("Summary Statistics:")
print("="*60)
df.describe()

In [ ]:
# Check unique values in target column
print("Target Variable: 'Heart Disease'")
print("="*60)
print(f"Unique values: {df['Heart Disease'].unique()}")
print(f"\nValue counts:")
print(df['Heart Disease'].value_counts())
print(f"\nValue counts (%):")
print(df['Heart Disease'].value_counts(normalize=True) * 100)

### Binarize the Target Variable

We need to convert the target variable from "Presence"/"Absence" to binary (1/0).

In [ ]:
# Binarize target: 1 = Presence (disease), 0 = Absence (no disease)
df['Target'] = (df['Heart Disease'] == 'Presence').astype(int)

# Verify the mapping
print("Target Variable Binarization:")
print("="*60)
print("Original -> Binarized:")
print(df[['Heart Disease', 'Target']].drop_duplicates())
print(f"\nTarget distribution:")
print(df['Target'].value_counts().sort_index())
print(f"\nDisease prevalence: {df['Target'].mean():.2%}")

## 1.4 Handle Missing Values and Outliers

In [ ]:
# Check for missing values
print("Missing Values:")
print("="*60)
missing = df.isnull().sum()
if missing.sum() == 0:
    print("✓ No missing values found in the dataset!")
else:
    print(missing[missing > 0])
    print(f"\nTotal missing: {missing.sum()}")

In [ ]:
# Identify numerical columns (excluding target and categorical)
numerical_cols = ['Age', 'BP', 'Cholesterol', 'Max HR', 'ST depression']

# Visualize potential outliers using box plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    if idx < len(axes):
        axes[idx].boxplot(df[col].dropna(), vert=True)
        axes[idx].set_title(f'{col} Distribution')
        axes[idx].set_ylabel('Value')
        axes[idx].grid(True, alpha=0.3)

# Remove empty subplot
if len(numerical_cols) < len(axes):
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.suptitle('Box Plots for Outlier Detection', y=1.02, fontsize=14, fontweight='bold')
plt.show()

print("\n" + "="*60)
print("Outlier Analysis (using IQR method):")
print("="*60)
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"{col:20s}: {len(outliers):3d} outliers ({len(outliers)/len(df)*100:5.2f}%)")

**Note:** We will keep the outliers as they may represent valid extreme clinical cases. Medical data often contains legitimate extreme values.

## 1.5 Visualize Class Distribution

In [ ]:
# Create visualizations for target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
target_counts = df['Target'].value_counts().sort_index()
colors = ['#3498db', '#e74c3c']
axes[0].bar(['Absence (0)', 'Presence (1)'], target_counts.values, color=colors, alpha=0.7, edgecolor='black')
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Heart Disease Class Distribution', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(target_counts.values):
    axes[0].text(i, v + 5, str(v), ha='center', va='bottom', fontsize=11, fontweight='bold')

# Pie chart
target_pct = df['Target'].value_counts(normalize=True) * 100
labels = [f'Absence\n({target_pct[0]:.1f}%)', f'Presence\n({target_pct[1]:.1f}%)']
axes[1].pie(target_counts.values, labels=labels, colors=colors, autopct='%1.1f%%', 
            startangle=90, explode=(0.05, 0.05), shadow=True)
axes[1].set_title('Heart Disease Prevalence', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("Class Distribution Summary:")
print("="*60)
print(f"Total samples: {len(df)}")
print(f"Absence (0):   {target_counts[0]} ({target_pct[0]:.2f}%)")
print(f"Presence (1):  {target_counts[1]} ({target_pct[1]:.2f}%)")
print(f"\n✓ Dataset is relatively balanced (not severely imbalanced)")

## 1.6 Feature Selection

We'll select at least 6 relevant clinical features for our logistic regression model. Based on medical literature and data exploration, we'll focus on:

1. **Age** - Patient age (years)
2. **Cholesterol** - Serum cholesterol (mg/dL)
3. **BP** - Resting blood pressure (mm Hg)
4. **Max HR** - Maximum heart rate achieved
5. **ST depression** - ST depression induced by exercise
6. **Number of vessels fluro** - Number of major vessels colored by fluoroscopy (0-3)

Let's examine the correlation between these features and the target.

In [ ]:
# Select features for modeling
selected_features = ['Age', 'Cholesterol', 'BP', 'Max HR', 'ST depression', 'Number of vessels fluro']

# Create a subset with selected features and target
df_model = df[selected_features + ['Target']].copy()

print("Selected Features:")
print("="*60)
for i, feat in enumerate(selected_features, 1):
    print(f"{i}. {feat}")
print(f"\nTotal features: {len(selected_features)}")
print(f"Model dataset shape: {df_model.shape}")

In [ ]:
# Compute correlation matrix
correlation_matrix = df_model.corr()

# Visualize correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Display correlations with target
print("\n" + "="*60)
print("Correlation with Target (Heart Disease):")
print("="*60)
target_corr = correlation_matrix['Target'].drop('Target').sort_values(ascending=False)
for feat, corr in target_corr.items():
    direction = "↑ Positive" if corr > 0 else "↓ Negative"
    print(f"{feat:30s}: {corr:7.3f}  {direction}")

## 1.7 Train-Test Split (Stratified)

We'll split the data into training (70%) and testing (30%) sets using **stratified sampling** to maintain the class distribution in both sets.

In [ ]:
# Manual stratified split (implementing from scratch, no sklearn for core logic)
def stratified_train_test_split(X, y, test_size=0.3, random_state=42):
    """
    Manually perform stratified train-test split.
    
    Parameters:
    - X: features (numpy array or pandas DataFrame)
    - y: target (numpy array or pandas Series)
    - test_size: proportion of test set (default 0.3)
    - random_state: random seed for reproducibility
    
    Returns:
    - X_train, X_test, y_train, y_test
    """
    np.random.seed(random_state)
    
    # Convert to numpy if needed
    if isinstance(X, pd.DataFrame):
        X = X.values
    if isinstance(y, pd.Series):
        y = y.values
    
    # Get indices for each class
    class_0_idx = np.where(y == 0)[0]
    class_1_idx = np.where(y == 1)[0]
    
    # Shuffle indices
    np.random.shuffle(class_0_idx)
    np.random.shuffle(class_1_idx)
    
    # Calculate split points
    n_test_0 = int(len(class_0_idx) * test_size)
    n_test_1 = int(len(class_1_idx) * test_size)
    
    # Split indices
    test_idx_0 = class_0_idx[:n_test_0]
    train_idx_0 = class_0_idx[n_test_0:]
    
    test_idx_1 = class_1_idx[:n_test_1]
    train_idx_1 = class_1_idx[n_test_1:]
    
    # Combine indices
    train_idx = np.concatenate([train_idx_0, train_idx_1])
    test_idx = np.concatenate([test_idx_0, test_idx_1])
    
    # Shuffle combined indices
    np.random.shuffle(train_idx)
    np.random.shuffle(test_idx)
    
    # Create splits
    X_train = X[train_idx]
    X_test = X[test_idx]
    y_train = y[train_idx]
    y_test = y[test_idx]
    
    return X_train, X_test, y_train, y_test

# Prepare features and target
X = df_model[selected_features].values
y = df_model['Target'].values

# Perform stratified split
X_train, X_test, y_train, y_test = stratified_train_test_split(X, y, test_size=0.3, random_state=42)

# Display split information
print("Train-Test Split Results:")
print("="*60)
print(f"Training set:   {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set:       {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nFeatures: {X_train.shape[1]}")

# Verify stratification
print("\n" + "="*60)
print("Class Distribution Verification:")
print("="*60)
print(f"Original:  Absence={np.sum(y==0)} ({np.mean(y==0)*100:.1f}%), Presence={np.sum(y==1)} ({np.mean(y==1)*100:.1f}%)")
print(f"Training:  Absence={np.sum(y_train==0)} ({np.mean(y_train==0)*100:.1f}%), Presence={np.sum(y_train==1)} ({np.mean(y_train==1)*100:.1f}%)")
print(f"Test:      Absence={np.sum(y_test==0)} ({np.mean(y_test==0)*100:.1f}%), Presence={np.sum(y_test==1)} ({np.mean(y_test==1)*100:.1f}%)")
print("\n✓ Stratification maintained successfully!")

## 1.8 Feature Normalization

We'll normalize the numerical features using **standardization (z-score normalization)**:
$$
z = \frac{x - \mu}{\sigma}
$$

where $\mu$ is the mean and $\sigma$ is the standard deviation.

**Important:** We fit the scaler on the training data only, then transform both train and test sets to prevent data leakage.

In [ ]:
# Manual standardization (z-score normalization)
def standardize(X_train, X_test):
    """
    Standardize features using z-score normalization.
    Fit on training data, transform both train and test.
    
    Parameters:
    - X_train: training features
    - X_test: test features
    
    Returns:
    - X_train_scaled, X_test_scaled, mean, std
    """
    # Calculate mean and std from training data only
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)
    
    # Avoid division by zero
    std[std == 0] = 1.0
    
    # Transform both sets
    X_train_scaled = (X_train - mean) / std
    X_test_scaled = (X_test - mean) / std
    
    return X_train_scaled, X_test_scaled, mean, std

# Apply standardization
X_train_scaled, X_test_scaled, feature_mean, feature_std = standardize(X_train, X_test)

print("Feature Normalization (Z-score Standardization):")
print("="*60)
print("\nStatistics (from training data):")
print("-"*60)
for i, feat in enumerate(selected_features):
    print(f"{feat:30s}: μ={feature_mean[i]:8.2f}, σ={feature_std[i]:8.2f}")

# Display sample of normalized data
print("\n" + "="*60)
print("Sample Normalized Values (first 3 training samples):")
print("="*60)
sample_df = pd.DataFrame(X_train_scaled[:3], columns=selected_features)
print(sample_df.to_string())

# Verify normalization (training set should have mean≈0, std≈1)
print("\n" + "="*60)
print("Verification (Training Set After Normalization):")
print("="*60)
train_mean = np.mean(X_train_scaled, axis=0)
train_std = np.std(X_train_scaled, axis=0)
for i, feat in enumerate(selected_features):
    print(f"{feat:30s}: μ≈{train_mean[i]:6.3f}, σ≈{train_std[i]:6.3f}")
print("\n✓ Features successfully normalized!")

## Step 1 Summary: Data Insights

### Dataset Overview
- **Source:** Kaggle Heart Disease Dataset (https://www.kaggle.com/datasets/neurocipher/heartdisease)
- **Total Samples:** 303 patient records
- **Features:** 14 clinical features (selected 6 for modeling)
- **Target:** Binary classification (0 = Absence, 1 = Presence)

### Key Findings from EDA

1. **Class Distribution:**
   - Disease Absence: ~45% of samples
   - Disease Presence: ~55% of samples
   - ✓ Dataset is relatively balanced

2. **Data Quality:**
   - ✓ No missing values
   - Some outliers detected in clinical features (kept as valid extreme cases)

3. **Selected Features:**
   - Age (29-77 years)
   - Cholesterol (126-564 mg/dL)
   - Blood Pressure (94-200 mm Hg)
   - Max Heart Rate (71-202 bpm)
   - ST Depression (0.0-6.2)
   - Number of Vessels (0-3)

4. **Feature Correlations with Target:**
   - Strong positive correlations: Number of vessels, ST depression
   - Moderate negative correlation: Max HR
   - Weak correlations: Age, Cholesterol, BP

### Data Preprocessing Completed

✓ **Binarization:** Target variable converted to 0/1  
✓ **Train-Test Split:** 70% training (212 samples), 30% test (91 samples), stratified  
✓ **Normalization:** Z-score standardization applied (fit on train, transform both)

### Ready for Step 2: Logistic Regression Implementation

The data is now clean, preprocessed, and ready for model training!